libraries that we need

In [1]:
import os
import zipfile
import pandas as pd
from collections import defaultdict
import re
import matplotlib.pyplot as plt

read parquet files and create a dataframe and save them in csv files

In [ ]:
import os
import pandas as pd
import re
from collections import defaultdict

def read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row):
    data_frames = defaultdict(list)
    group_names = [
        'home_team_score',
        'home_team',
        'away_team',
        'away_team_score',
        'event',
        'round',
        'season',
        'time',
        'tournament',
        'venue',
        'odds',
        'pbp',
        'statistics',
        'power',
        'votes'
    ]

    # Regular expressions for away_team and away_team_score
    away_team_pattern = re.compile(r'^away_team_')
    away_team_score_pattern = re.compile(r'^away_team_score_')

    for date in range(start_date, end_date + 1):
        for folder in folders_in_row:
            date_folder = os.path.join(base_folder, str(date), 'data', 'raw', folder)
            print(f'Reading folder: {date_folder}')  # Monitor the current folder
            if os.path.isdir(date_folder):
                for file_name in os.listdir(date_folder):
                    if file_name.endswith('.parquet'):
                        matched_group = None
                        # Check each group name if the filename starts with it
                        if away_team_score_pattern.match(file_name):
                            matched_group = 'away_team_score'
                        elif away_team_pattern.match(file_name):
                            matched_group = 'away_team'
                        else:
                            # Simple string matching for other group names
                            for group_name in group_names:
                                if file_name.startswith(group_name + '_'):
                                    matched_group = group_name
                                    break

                        if matched_group:
                            file_path = os.path.join(date_folder, file_name)
                            df = pd.read_parquet(file_path)
                            data_frames[matched_group].append(df)

    # Concatenate DataFrames in each group and save to CSV
    grouped_data_frames = {}
    for group, dfs in data_frames.items():
        if dfs:  # Check if the list of DataFrames is not empty
            concatenated_df = pd.concat(dfs, ignore_index=True)
            grouped_data_frames[group] = concatenated_df
            # Save DataFrame to CSV
            csv_file_path = os.path.join(base_folder, f'{group}.csv')
            concatenated_df.to_csv(csv_file_path, index=False)
            print(f'Saved {group} DataFrame to {csv_file_path}')

    return grouped_data_frames

# Example usage
base_folder = r'F:\daneshkar\project1\extracted_rar_files'
start_date = 20240501
end_date = 20240531
folders_in_row = (
    'raw_match_parquet',
    'raw_odds_parquet',
    'raw_point_by_point_parquet',
    'raw_statistics_parquet',
    'raw_tennis_power_parquet',
    'raw_votes_parquet'
)

grouped_data_frames = read_and_group_parquet_files(base_folder, start_date, end_date, folders_in_row)

# Display the groups and their DataFrames
for group_name, df in grouped_data_frames.items():
    print(f'Group: {group_name}')
    # print(df.head())  # Print the first few rows of each DataFrame
